In [1]:
from modelTrainer import ModelTrainer
from metrics import RMSE, percentError

/home/ubuntu/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


### Common Variables

In [2]:
# path to data on aws
u = "/home/ubuntu/project/"
data_path = u + "data/groundcover2016/"
model_path = u + "model_checkpoints/unet"
results_path = u + "results"
# create model trainer
trainer = ModelTrainer(data_path,results_path,model_path)
trainer.setRegression()
trainer.changeMetrics([RMSE,percentError])

### Convert Single class model and do more training

In [5]:
trainer.setOldModel('smallNetwork2.hdf5')
trainer.changeEpochs(1)
trainer.changeBatchSize(256)
trainer.changeDropout(0)
trainer.setOptimizerParams(1.0*(10**-6),0.9,1.0*(10**-10))
trainer.setClassMap({'maize': 1})
trainer.setSaveName('smallSingleClassNetworkRegression')
trainer.train()

----------Model Parameters----------
Initial Conv. Depth : 4
Number of Classes   : 2
Dropout             : 0
Activation Function : Relu
Input Shape         : [256, 256, 3]
Batch Size          : 256
--------Optimizer Parameters--------
Learning Rate : <tf.Variable 'SGD_4/lr:0' shape=() dtype=float32_ref>
Momentum      : <tf.Variable 'SGD_4/momentum:0' shape=() dtype=float32_ref>
Initial Decay : 1e-10
Epoch 1/1
 11/278 [>.............................] - ETA: 16:57 - loss: 0.6527 - RMSE: 0.8078 - percentError: -inf

TypeError: unsupported operand type(s) for +: 'NoneType' and 'NoneType'

In [ ]:
# fit the model
model.fit_generator(generator = trainGenerator,
                    validation_data = validateGenerator,
                    steps_per_epoch = steps_per_epoch,
                    validation_steps = validation_steps,
                    epochs = epochs,
                    use_multiprocessing = True,
                    callbacks =[model_checkpoint, logger])

Epoch 1/5
125/278 [============>.................] - ETA: 7:08 - loss: 2144686843.9040 - acc: 0.8418